In [1]:
import cascade
from cascade import PyPoint
import pandas as pd
import time
import random
import string

def random_string(length=5):
    # Generate a random string of a given length (default: 5 characters)
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))

In [2]:
gtfs_path = "/home/chingiz/Rust/py_rust/cascade/cascade-bin/files/Saint_Petersburg"
pbf_path = "/home/chingiz/Rust/osm/roads_SZ.pbf"
departure = 0
duration = 90000
weekday = "monday"

graph = cascade.create_graph(gtfs_path, pbf_path, departure, duration, weekday)

Filtering left 1942842 rows
number of edges 1439514
1
517512
Graph creation time: 16.420134631s


In [3]:
mapping = graph.get_mapping()
len(mapping)

526089

In [4]:
transit = 0
walk = 0

for key, value in mapping.items():
    if value.get_node_type() == "street":
        walk += 1
    else:
        transit += 1

In [5]:
print(transit, walk)

8577 517512


In [18]:
list_of_points = []
# Define the bounding box
bbox = {
    "min_x": 61.331478,
    "max_x": 61.438925,
    "min_y": 55.132710,
    "max_y": 55.196110
}

bbox = {
    "min_x": 30.349061,
    "max_x": 30.370268,
    "min_y": 59.878163,
    "max_y": 59.851074
}
 
# Generate 1000 random points within the bounding box
random_points = [
    PyPoint(
        random.uniform(bbox["min_x"], bbox["max_x"]),
        random.uniform(bbox["min_y"], bbox["max_y"]),
        random_string(7) 
    )
    for _ in range(1000)
]

list_of_points.extend(random_points)

In [19]:
mapping = {p.id():(p.y(),p.x()) for p in list_of_points}

In [20]:
start = time.perf_counter()
result = cascade.calculate_od_matrix(graph, list_of_points, 43200)
print("Time elapsed", time.perf_counter() - start, len(result))

# crate dataframe from result dict of dict
df: pd.DataFrame = pd.DataFrame.from_dict(result, orient="index").fillna(0)

df_long = df.stack().reset_index()
df_long.columns = ['from_point', 'to_point', 'weight']

Time elapsed 531.798696326 10000


In [21]:
df_long["coord_source"] = df_long["from_point"].apply(lambda x: mapping.get(x))
df_long["coord_target"] = df_long["to_point"].apply(lambda x: mapping.get(x))

In [22]:
df_long

,from_point,to_point,weight,coord_source,coord_target
0,fv6ueXg,6oGYNwx,627.499264,"(59.87460615394942, 30.358036549373328)","(59.871411303658185, 30.35465764979515)"
1,fv6ueXg,L7vneme,2013.238040,"(59.87460615394942, 30.358036549373328)","(59.858025396776945, 30.351878619582614)"
2,fv6ueXg,BvyZ0om,1051.432667,"(59.87460615394942, 30.358036549373328)","(59.86529636088742, 30.363269149555467)"
3,fv6ueXg,fBn8zEf,866.148965,"(59.87460615394942, 30.358036549373328)","(59.86710471260182, 30.36788056494167)"
4,fv6ueXg,IlVRyGG,2135.838727,"(59.87460615394942, 30.358036549373328)","(59.857697738999256, 30.358845198738017)"
...,...,...,...,...,...
32379995,7agGsJY,4iApKef,1826.550414,"(59.8589273104206, 30.360032828418007)","(59.87287956893644, 30.34934161073305)"
32379996,7agGsJY,UpryVn8,866.205683,"(59.8589273104206, 30.360032828418007)","(59.85600473689304, 30.349531822317612)"
32379997,7agGsJY,auReBik,1654.380693,"(59.8589273104206, 30.360032828418007)","(59.86034365857978, 30.367629862517063)"
32379998,7agGsJY,WU2PMOf,2338.865999,"(59.8589273104206, 30.360032828418007)","(59.86964659547507, 30.367747606273415)"


In [23]:
mapping = graph.get_mapping()
print(mapping[226386].get_geometry())
print(mapping[403891].get_geometry())

(30.314132999999998, 59.9112029)
(29.8157905, 59.8721305)


In [24]:
dd = graph.get_mapping()
dd[398264].get_geometry()

(30.2687346, 59.8907271)

In [25]:
print(len(cascade.single_source_shortest_path(graph, 43200, 30.320234, 59.875912)))
(source_x, source_y)  = [(y,x) for (x,y) in [((59.89664126900218, 30.342734328605864))]][0]
(target_x, target_y) = [(y,x) for (x,y) in [(59.90557796210815, 30.417929109882287)]][0]

start = time.perf_counter()
print(
    cascade.shortest_path(
        graph=graph,
        dep_time=43200,
        source_x=source_x,
        source_y=source_y,
        target_x=target_x,
        target_y=target_y,
    )
)
print("Time elapsed", time.perf_counter() - start)

526089
3804.3809961511206
Time elapsed 0.08361929300008342


: 